In [4]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt 
import numpy as np



import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import Ridge 
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV


import xgboost as xgb
from xgboost import XGBRegressor
from ydata_profiling import ProfileReport


from scalecast.Forecaster import Forecaster
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## Import Data

In [5]:
train = pd.read_csv('train.csv')

In [6]:
data_profile = ProfileReport(train)

In [7]:
def feature_eng(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Extract date features
    df['day_of_year'] = df['date'].dt.dayofyear
    df['day_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

    # Encode categorical features
    le = LabelEncoder()
    df['country'] = le.fit_transform(df['country'])
    df['store'] = le.fit_transform(df['store'])
    df['product'] = le.fit_transform(df['product'])

    # Drop missing values and irrelevant columns
    df = df.dropna()
    df = df.set_index('date')
    df = df.drop(columns=['id'])  # Dropping columns that are not useful for modeling
    # Separate features and target
    X = df.drop(columns=['num_sold'])
    y = df['num_sold']

    # Scale features
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return df, X,  y

# Feature engineering
df, X, y = feature_eng(train)

# Splitting the data
SPLIT = 0.85
split_index = int(SPLIT * len(X))

X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (188070, 7)
y_train shape: (188070,)
X_test shape: (33189, 7)
y_test shape: (33189,)


In [8]:
df.head()

,country,store,product,num_sold,day_of_year,day_week,month,year
date,,,,,,,,
2010-01-01,0,0,1,973.0,1,4,1,2010
2010-01-01,0,0,2,906.0,1,4,1,2010
2010-01-01,0,0,3,423.0,1,4,1,2010
2010-01-01,0,0,4,491.0,1,4,1,2010
2010-01-01,0,2,0,300.0,1,4,1,2010


In [9]:
X_train

,country,store,product,day_of_year,day_week,month,year
date,,,,,,,
2010-01-01,0,0,1,1,4,1,2010
2010-01-01,0,0,2,1,4,1,2010
2010-01-01,0,0,3,1,4,1,2010
2010-01-01,0,0,4,1,4,1,2010
2010-01-01,0,2,0,1,4,1,2010
...,...,...,...,...,...,...,...
2015-12-15,4,2,1,349,1,12,2015
2015-12-15,4,2,2,349,1,12,2015
2015-12-15,4,2,3,349,1,12,2015


In [ ]:

xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
param_dist = {
    'n_estimators': [30, 50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.15,0.2],
    'max_depth': [3, 6, 9, 18,36],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    scoring='neg_mean_absolute_percentage_error',
    n_iter=10,
    cv=3,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [10]:
y_pred = random_search.best_estimator_.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Optimized Test MAPE: {mape:.4f}")

Optimized Test MAPE: 0.0957
